In [ ]:
import pandas as pd

fake = pd.read_csv("Fake.csv", engine="python", on_bad_lines="skip", encoding="latin1", quoting=3)
true = pd.read_csv("True.csv", engine="python", on_bad_lines="skip", encoding="latin1", quoting=3)

fake['label'] = 0

true['label'] = 1

df = pd.concat([fake, true]).reset_index(drop=True)
df.head()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(6,4))
sns.countplot(x=df['label'])
plt.title("Number of Fake vs Real News Articles")
plt.show()


In [ ]:
df['text'] = df['text'].fillna("")

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

X = df['text']
y = df['label']

tfidf = TfidfVectorizer(stop_words='english', max_features=5000)
X_tfidf = tfidf.fit_transform(X)


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=200)
model.fit(X_train, y_train)


In [ ]:
def predict_news(text):
    text_tfidf = tfidf.transform([text])
    prediction = model.predict(text_tfidf)[0]
    return "REAL NEWS" if prediction == 1 else "FAKE NEWS"

predict_news("The president announced a new economic reform today.")


In [ ]:
import pickle

# Save model
with open("model.pkl", "wb") as f:
    pickle.dump(model, f)

# Save TF-IDF vectorizer
with open("tfidf.pkl", "wb") as f:
    pickle.dump(tfidf, f)


In [ ]:
%%writefile app.py
import streamlit as st
import pickle

with open("model.pkl", "rb") as f:
    model = pickle.load(f)

with open("tfidf.pkl", "rb") as f:
    tfidf = pickle.load(f)

st.title("📰 Fake News Detection App")
st.write("Enter any news article or headline below to check if it's **Real** or **Fake**.")

user_input = st.text_area("Paste News Text Here:", height=200)

if st.button("Predict"):
    if user_input.strip() == "":
        st.warning("Please enter some text before predicting.")
    else:
        input_tfidf = tfidf.transform([user_input])
        prediction = model.predict(input_tfidf)[0]

        if prediction == 1:
            st.success("This news is **REAL**.")
        else:
            st.error("This news is **FAKE**.")


In [ ]:
!python -m streamlit run app.py

In [ ]:
# Fake News for testing
# "Fox News Bans Gene Simmons For Life For Harassing Staff Off Camera"
# Real News for testing
# "The president announced a new economic reform today"
    